In [12]:
%display latex
import code as pc
import numpy as np
Partitions.options.latex='list'

# to_fock ($\lambda$):

Takes a partition $$\lambda = \lambda_1 + \cdots + \lambda_\ell$$ of $n$, returns the fock vector $\mathbf{v}$ which satisfies $$\mathbf{v}_i = \lambda_i - i.$$

*(Note that $i+1$ is needed everywhere due to Python's 0-based indexing.)*

# to_partition ($\mathbf{v}$):

Takes a fock vector $\mathbf{v}$ and returns the partition $\lambda$ satisfying the above condition.

In [13]:
# Given a partition 
def to_fock(partition):
    return tuple([(x-(i+1)) for i, x in enumerate(partition)])

def to_fock_singleton(partition):
    v = to_fock(partition)
    return([(1, v)])

def to_partition(fock_vector):
    return Partition([(x+(i+1)) for i, x in enumerate(fock_vector)])

# num_swaps (L):

Takes a list $L$, and produces a sorted list $L'$ that is obtained from $L$ by using only transpositions. Also keeps track of $s$, the number of transpositions needed, and produces the following sign:
$$
\mathrm{sign}(s) = \begin{cases}-1 & \text{if } s \text{ is even} \\ +1 & \text{else} \end{cases}
$$

In [31]:
# Take a list, sort it into weakly decreasing order using only
# transpositions, and count the number of transpositions needed.
def num_swaps(L):
    Lp = copy(L)
    swapcount = 0
    for j in range(len(Lp)):
        for i in range(1, len(Lp)-j):
            if Lp[i-1] > Lp[i]:
                swapcount += 1
                Lp[i-1], Lp[i] = Lp[i], Lp[i-1]
    sign = 1 if swapcount % 2 == 0 else -1
    Lpp = list(Lp)
    Lpp.reverse()
    return sign, tuple(Lpp)

# $\alpha_k(\mathbf{v}_\lambda)$

Given a fock vector $\mathbf{v}_\lambda = [v_1, v_2, \cdots, v_\ell]$, first computes the application
$$
\begin{align*}
v_1 + \mathbf{k} &\wedge v_2     &\wedge \cdots &\wedge v_\ell \\
v_1     &\wedge v_2 + \mathbf{k} &\wedge \cdots &\wedge v_\ell \\
v_1 &\wedge v_2 &\wedge \cdots &\wedge v_\ell + \mathbf{k}
\end{align*}
$$

We do this by stacking $\mathbf{v}$ as the rows of a matrix, and subtracting the scalar matrix $kI$.

We then proceed to wedge each summand above back into weakly decreasing order. 

- Noting that any summand with a repeated entry will wedge to zero, we delete it.

- We use the *num_swaps* function to sort each vector using transpositions, tracking the sign of the transpositions.


In [49]:
def vacuum(n):
    return([-i-1 for i in range(n)])

def partial_vacuum(n, k):
    l1 = vector(vaccum(n))
    l2 = vector(vaccum(k) + [ZZ(0)]*(n-k))
    return((l1 - l2))

def alpha(k, v_lambda):
    
    # Length of fock vector
    n = len(v_lambda)
                
    v_lambda
    # Stack n copies of v_lambda up as the rows of an nxn matrix
    V = Matrix([ v_lambda for i in range(n) ])
    
    # A scalar matrix for k
    kI = identity_matrix(n) * k
    
    # Subtract k off of the diagonal
    M = V - kI
    
    # Sorted vectors, with the (signs of the) number of transpositions
    # Throw away vectors with repeated entries, since these
    # wedge to zero.
    L = [num_swaps(R) for R in M.rows() if len(R) == len(set(R))]

    return(L)

def distribute(lin_comb):
    outs = []
    for vs in lin_comb:
        c0 = vs[0]
        for v in vs[1]:
            ci, vi = v
            outs.append((ci * c0, vi ))
    return(outs)

def collect_terms(list_of_vs):
    D = dict()
    for i, x in enumerate(list_of_vs):
        c_i, v = x
        D[v] = c_i if v not in D.keys() else D[v] + c_i
    return([(D[i], i) for i in D if D[i] != 0])

def alpha_bar(k, ls):
    lin_comb = [(ci, alpha(k, vi)) for ci, vi in ls]
    reduced = distribute(lin_comb)
    more_reduced = collect_terms(reduced)
    return(more_reduced)

In [42]:
v_l = to_fock_singleton((3,1,1))
show(v_l)

[(1, (2, -1, -2))]

In [48]:
a2 = alpha(-2, v_l[0][1])
a2

[(-1, (4, -1, -2)), (-1, (2, 1, -2)), (1, (2, 0, -1))]

In [44]:
a2 = alpha(0, v_l[0][1])
a2

[(-1, (2, -1, -2)), (-1, (2, -1, -2)), (-1, (2, -1, -2))]

In [52]:
alpha_bar(-2, v_l)

[(-1, (4, -1, -2)), (-1, (2, 1, -2)), (1, (2, 0, -1))]

In [53]:
a3 = alpha_bar(3, a2)
a3

[(-1, (4, -2, -4)),
 (1, (4, -1, -5)),
 (1, (2, 1, -5)),
 (1, (2, -1, -3)),
 (-1, (2, 0, -4))]

# Notes from Monday Feb 24th

> DZG: From conversation with Richard earlier!

Note that $\alpha_k$ only works on a single basis fock vector $\mathbf{v}_\lambda$ -- in general, it should operate on a formal linear combination of fock vectors $\mathbf{v} = \sum_{i=1}^n c_i \mathbf{v}_i$.

How to do this? Idea: represent $\mathbf v = \sum_{i=1}^n c_i \mathbf{v}_i$ as a list of pairs 
$$
\mathbf v = [(c_1, \mathbf{v}_1), (c_2, \mathbf{v}_2), \cdots, (c_n, \mathbf{v}_n)]
$$

It's then easy to set
$$
\alpha_k(\mathbf v) = 
[(c_1, \alpha_k(\mathbf{v}_1) ), (c_2, \alpha_k(\mathbf{v}_2)), \cdots, (c_n, \alpha_k(\mathbf{v}_n))]
$$

Note that in each pair, the second entry is now *itself* a list of pairs. In gory detail, we have
$$
\alpha_k(\mathbf v) = 
[~(c_1, ~[ (d_{11}, \mathbf{v}_{11}), (d_{12}, \mathbf{v}_{12}), \cdots ]~ ), ~\cdots~, (c_n,  ~[ (d_{n1}, \mathbf{v}_{n1}), (d_{n2}, \mathbf{v}_{n2}), \cdots ]~ )~]
$$

In [ ]:
var('z')
sun = 1/(e^(z/2)-e^(-z/2))
def pk(k, partition):
    fock_vector = to_fock(partition)
    if k%2==0:
        ck = 0
    if k%2==1:
        ck = (sun.taylor(z,0,k+1)).coefficient(z^k)/k.factorial()
    return(ck + sum((x+1/2)^k-(-i+1/2)^k for i,x in enumerate(fock_vector))/k.factorial())
print(pk(5,(3,2,1,1)))

In [ ]:
def chi(representation, cycle_type):
    for i in range(size(representation)-size(cycle_type)):
        cycle_type.append(1)
    fock_vector = to_fock(representation)
    for i in range(len(cycle_type)):
        fock_vector = alpha(cycle_type[i], fock_vector)
    return(fock_vector[1])

In [ ]:
def hook_lengths(partition):
    return(list of hook lengths)

In [ ]:
def dim(partition):
    return(size(partition)!/prod(hook_lengths(partition)))

In [ ]:
def conj_class_size(partition):
    return(size(partition)!/(prod(partition)*prod(numbers of times i appears!)))

In [ ]:
def eff(representation, cycle_type):
    return(chi(representation, cycle_type)*conj_class_size(cycle_type)/dim(partition))

In [ ]:
def pillowcase_weight(N,partition):
    (compute sign somehow)
    return sign*product(hook_lengths s.t. hook_length%N = pm 1)/prod(hook_lengths s.t. hook_length%N=0)^2

In [ ]:
def quotient_partitions(N,partition):

In [ ]:
def core_partition(N,partition):